In [249]:
import numpy as np
import pandas as pd
import xgboost as xgb
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder

In [250]:
df = pd.read_csv('../udataset/wealth_data.csv') # Using meat production as example

unemployment_data_df = pd.read_csv('../udataset/unemployment_by_state.csv')

health_data_df = pd.read_csv('../udataset/health_data.csv')

commodity_data_df = pd.read_csv('../udataset/all_commodities.csv')

meat_df = pd.read_csv('../udataset/meat/production.csv')
meat_df = meat_df[meat_df['DateTime'].str.contains(r'-12')]
meat_df = meat_df.rename(columns={'DateTime': 'year'})
meat_df['year'] = meat_df['year'].str.slice(0, 4)
meat_df['year'] = meat_df['year'].astype(int)



unemployment_data_df = unemployment_data_df.rename(columns={'State': 'state'})

#unemployment_data_df.drop(columns=['YearMonth'], inplace=True)
unemployment_data_df = unemployment_data_df[unemployment_data_df['YearMonth'].str.contains(r'-12')]
unemployment_data_df = unemployment_data_df.rename(columns={'YearMonth': 'year'})
unemployment_data_df['year'] = unemployment_data_df['year'].str.slice(0, 4)
unemployment_data_df['year'] = unemployment_data_df['year'].astype(int)

#health_data_df.drop(columns=['year'], inplace=True)
health_data_df['state'] = health_data_df['state'].str.lower()

commodity_data_df.drop(columns=['date'], inplace=True)
# commodity_data_df['year'] = pd.to_datetime(commodity_data_df['date']).dt.year
# commodity_data_df.drop(columns = ['date'], inplace=True)

# df = df.merge(unemployment_data_df, on=['year', 'state'], how='left')
# df = df.merge(health_data_df, on=['year', 'state'], how='left')
# df = df.merge(meat_df, on = ['year'], how='left')

print(df.info())

# TODO - Charles u sure we don't need time series data?
# df = df.merge(commodity_data_df, on='year', how='left')


#df = df.drop(columns=['below_poverty_line_percent'])
df = df.drop(columns=['Unnamed: 0'])
df = df[df['year'] > 2010]
print(df.shape[1])
model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)



# def format(df):
#     for col in df.columns:
#         df[f'tmp_{col}'] = df[col].shift(1)

#     return df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 6 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    676 non-null    int64  
 1   state                                         676 non-null    object 
 2   year                                          676 non-null    int64  
 3   below_poverty_line_percent                    676 non-null    float64
 4   median_household_income                       676 non-null    float64
 5   mean cash public assistance income (dollars)  676 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 31.8+ KB
None
5


In [251]:
health_data_df

,state,year,obesity_adult_rate,overweight_adult_rate,adults_no_leisure-time_physical_activity
0,alaska,2011,0.346154,0.98,0.177378
1,alaska,2012,0.264423,1.00,0.087404
2,alaska,2013,0.394231,0.86,0.185090
3,alaska,2014,0.456731,0.60,0.105398
4,alaska,2015,0.461538,0.83,0.177378
...,...,...,...,...,...
643,oregon,2018,0.466346,0.51,0.107969
644,oregon,2019,0.423077,0.58,0.226221
645,oregon,2020,0.379808,0.71,0.087404
646,oregon,2021,0.490385,0.56,0.123393


In [252]:
train_df = df[df['year'] < 2021]
test_df = df[df['year'] >= 2021]

train_data = train_df.to_numpy()
train_labels = train_data[:, 2]
test_data = test_df.to_numpy()
test_labels = test_data[:, 2]

train_df = train_df.drop(columns = ['below_poverty_line_percent'])
test_df = test_df.drop(columns = ['below_poverty_line_percent'])
train_df = train_df.drop(columns=['year'])
test_df = test_df.drop(columns=['year'])

train_data = train_df.to_numpy()
test_data = test_df.to_numpy()
train_data[:, 0]
train_data.shape
train_labels.shape

(520,)

In [253]:
le = LabelEncoder()
le.fit(train_data[:, 0])
train_data[:, 0] = le.transform(train_data[:, 0])
train_data.shape

(520, 3)

In [254]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, train_data, train_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [255]:
scores = absolute(scores)
print(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

[0.58955855 0.68639687 0.68495773 0.69893391 0.76455996 0.57461576
 0.62550795 0.71124764 0.68051304 0.61828902 0.67416664 0.6703666
 0.56713297 0.6433572  0.85623474 0.51565962 0.59044495 0.62324455
 0.64283147 0.58606527 0.64260835 0.72937593 0.61462023 0.56290618
 0.56894083 0.53138096 0.5444811  0.67660394 0.67699589 0.56498316]
Mean MAE: 0.637 (0.073)
